<a href="https://colab.research.google.com/github/guyyariv/AI-Beat-Maker/blob/master/LO-OVED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup Environment
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
# !pip install -Iv https://pypi.python.org/packages/source/p/pyFluidSynth/pyFluidSynth-1.2.4.tar.gz#md5=60079310701d0b9298b65a8b6728ffcc
!pip install -U -q magenta
!gsutil -q -m cp gs://magentadata/soundfonts/Standard_Drum_Kit.sf2 .
!gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/groovae_*.tar .
!gsutil -q -m cp gs://magentadata/models/music_vae/groovae_colab/*wav .
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta
!git clone https://github.com/bearpelican/musicautobot.git
!pip install torch fastai music21 pebble fluidsynth midi2audio
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library  
import copy, warnings, librosa, numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Colab/Notebook specific stuff
import IPython.display
from IPython.display import Audio
from google.colab import files

# Magenta specific stuff
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.models.music_vae import data
import note_seq
from note_seq import midi_synth
from note_seq.sequences_lib import concatenate_sequences
from note_seq.protobuf import music_pb2



import magenta

import tensorflow
import os
os.chdir('musicautobot')

from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.setup_musescore import setup_musescore
setup_musescore()
from midi2audio import FluidSynth
from pydub import AudioSegment


from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2


import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"


import magenta.music as mm

import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings

In [6]:
# Load some configs to be used later
dc_quantize = configs.CONFIG_MAP['groovae_2bar_humanize'].data_converter
dc_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'].data_converter
dc_hihat = configs.CONFIG_MAP['groovae_2bar_add_closed_hh'].data_converter
dc_4bar = configs.CONFIG_MAP['groovae_4bar'].data_converter

GROOVAE_4BAR = "groovae_4bar.tar"
GROOVAE_2BAR_HUMANIZE = "/content/groovae_2bar_humanize.tar"
GROOVAE_2BAR_HUMANIZE_NOKL = "/content/groovae_2bar_humanize_nokl.tar"
GROOVAE_2BAR_HITS_CONTROL = "/content/groovae_2bar_hits_control.tar"
GROOVAE_2BAR_TAP_FIXED_VELOCITY = "/content/groovae_2bar_tap_fixed_velocity.tar"
GROOVAE_2BAR_ADD_CLOSED_HH = "/content/groovae_2bar_add_closed_hh.tar"
GROOVAE_2BAR_HITS_CONTROL_NOKL = "/content/groovae_2bar_hits_control_nokl.tar"

In [7]:
# Define some functions

# If a sequence has notes at time before 0.0, scootch them up to 0
def start_notes_at_0(s):
  for n in s.notes:
    if n.start_time < 0:
      n.end_time -= n.start_time
      n.start_time = 0
  return s

def play(note_sequence, sf2_path='Standard_Drum_Kit.sf2'):  
  if sf2_path:
    audio_seq = midi_synth.fluidsynth(start_notes_at_0(note_sequence), sample_rate=44100, sf2_path=sf2_path)
    IPython.display.display(IPython.display.Audio(audio_seq, rate=44100))
  else:
    note_seq.play_sequence(start_notes_at_0(note_sequence), synth=note_seq.fluidsynth)

# Some midi files come by default from different instrument channels
# Quick and dirty way to set midi files to be recognized as drums
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True
    
def unset_to_drums(ns):
  for note in ns.notes:
    note.is_drum=False
    note.instrument=0
  return ns

# quickly change the tempo of a midi sequence and adjust all notes
def change_tempo(note_sequence, new_tempo):
  new_sequence = copy.deepcopy(note_sequence)
  ratio = note_sequence.tempos[0].qpm / new_tempo
  for note in new_sequence.notes:
    note.start_time = note.start_time * ratio
    note.end_time = note.end_time * ratio
  new_sequence.tempos[0].qpm = new_tempo
  return new_sequence

def download(note_sequence, filename):
  note_seq.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)
  
def download_audio(audio_sequence, filename, sr):
  librosa.output.write_wav(filename, audio_sequence, sr=sr, norm=True)
  files.download(filename)


# quick method for removing microtiming and velocity from a sequence
def get_quantized_2bar(s, velocity=0):
  new_s = dc_quantize.from_tensors(dc_quantize.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  if velocity != 0:
    for n in new_s.notes:
      n.velocity = velocity
  return new_s

# quick method for turning a drumbeat into a tapped rhythm
def get_tapped_2bar(s, velocity=85, ride=False):
  new_s = dc_tap.from_tensors(dc_tap.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  if velocity != 0:
    for n in new_s.notes:
      n.velocity = velocity
  if ride:
    for n in new_s.notes:
      n.pitch = 42
  return new_s

# quick method for removing hi-hats from a sequence
def get_hh_2bar(s):
  new_s = dc_hihat.from_tensors(dc_hihat.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  return new_s


# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
  return note_seq.sequences_lib.quantize_note_sequence(s,steps_per_quarter)

# Destructively quantize a midi sequence
def flatten_quantization(s):
  beat_length = 60. / s.tempos[0].qpm
  step_length = beat_length / 4#s.quantization_info.steps_per_quarter
  new_s = copy.deepcopy(s)
  for note in new_s.notes:
    note.start_time = step_length * note.quantized_start_step
    note.end_time = step_length * note.quantized_end_step
  return new_s

# Calculate how far off the beat a note is
def get_offset(s, note_index):
  q_s = flatten_quantization(quantize(s))
  true_onset = s.notes[note_index].start_time
  quantized_onset = q_s.notes[note_index].start_time
  diff = quantized_onset - true_onset
  beat_length = 60. / s.tempos[0].qpm
  step_length = beat_length / 4#q_s.quantization_info.steps_per_quarter
  offset = diff/step_length
  return offset

def is_4_4(s):
  ts = s.time_signatures[0]
  return (ts.numerator == 4 and ts.denominator ==4)

def preprocess_4bar(s):
  return dc_4bar.from_tensors(dc_4bar.to_tensors(s).outputs)[0]

def preprocess_2bar(s):
  return dc_quantize.from_tensors(dc_quantize.to_tensors(s).outputs)[0]

def _slerp(p0, p1, t):
  """Spherical linear interpolation."""
  omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)),
    np.squeeze(p1/np.linalg.norm(p1))))
  so = np.sin(omega)
  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

In [ ]:
dataset_2bar = tfds.as_numpy(tfds.load(name="groove/2bar-midionly", split=tfds.Split.VALIDATION, try_gcs=True))
dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]
dataset_4bar = tfds.as_numpy(tfds.load(name="groove/4bar-midionly", split=tfds.Split.VALIDATION, try_gcs=True))
dev_sequences_4bar = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_4bar]
_ = [set_to_drums(s) for s in dev_sequences_4bar]
dev_sequences_4bar = [s for s in dev_sequences_4bar if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]

music_vae = TrainedModel(configs.CONFIG_MAP['cat-mel_2bar_big'], batch_size=4, checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')
note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')
bundle = sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

In [ ]:
import copy
from copy import deepcopy

config_2bar_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity']
groovae_2bar_tap = TrainedModel(config_2bar_tap, 1, checkpoint_dir_or_path=GROOVAE_2BAR_TAP_FIXED_VELOCITY)

def mix_tracks(y1, y2, stereo = False):
  l = max(len(y1),len(y2))
  y1 = librosa.util.fix_length(y1, l)
  y2 = librosa.util.fix_length(y2, l)
  
  if stereo:
    return np.vstack([y1, y2])  
  else:
    return y1+y2

def make_click_track(s):
  last_note_time = max([n.start_time for n in s.notes])
  beat_length = 60. / s.tempos[0].qpm 
  i = 0
  times = []
  while i*beat_length < last_note_time:
    times.append(i*beat_length)
    i += 1
  return librosa.clicks(times)

def drumify(s, model, temperature=1.0): 
  encoding, mu, sigma = model.encode([s])
  decoded = model.decode(encoding, length=32, temperature=temperature)
  return decoded[0]

def combine_sequences(seqs):
  # assumes a list of 2 bar seqs with constant tempo
  for i, seq in enumerate(seqs):
    shift_amount = i*(60 / seqs[0].tempos[0].qpm * 4 * 2)
    if shift_amount > 0:
      seqs[i] = note_seq.sequences_lib.shift_sequence_times(seq, shift_amount)
  return note_seq.sequences_lib.concatenate_sequences(seqs)

def combine_sequences_with_lengths(sequences, lengths):
  seqs = copy.deepcopy(sequences)
  total_shift_amount = 0
  for i, seq in enumerate(seqs):
    if i == 0:
      shift_amount = 0
    else:
      shift_amount = lengths[i-1]
    total_shift_amount += shift_amount
    if total_shift_amount > 0:
      seqs[i] = note_seq.sequences_lib.shift_sequence_times(seq, total_shift_amount)
  combined_seq = music_pb2.NoteSequence()
  for i in range(len(seqs)):
    tempo = combined_seq.tempos.add()
    tempo.qpm = seqs[i].tempos[0].qpm
    tempo.time = sum(lengths[0:i-1])
    for note in seqs[i].notes:
      combined_seq.notes.extend([copy.deepcopy(note)])
  return combined_seq

def get_audio_start_time(y, sr):
  tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
  beat_times = librosa.frames_to_time(beat_frames, sr=sr)
  onset_times = librosa.onset.onset_detect(y, sr, units='time')
  start_time = onset_times[0] 
  return start_time

def audio_tap_to_note_sequence(f, velocity_threshold=30):
  y, sr = librosa.load(f)
  # pad the beginning to avoid errors with onsets right at the start
  y = np.concatenate([np.zeros(1000),y])
  tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
  # try to guess reasonable tempo
  beat_times = librosa.frames_to_time(beat_frames, sr=sr)
  onset_frames = librosa.onset.onset_detect(y, sr, units='frames')
  onset_times = librosa.onset.onset_detect(y, sr, units='time')
  start_time = onset_times[0]
  onset_strengths = librosa.onset.onset_strength(y, sr)[onset_frames]
  normalized_onset_strengths = onset_strengths / np.max(onset_strengths)
  onset_velocities = np.int32(normalized_onset_strengths * 127)
  note_sequence = music_pb2.NoteSequence()
  note_sequence.tempos.add(qpm=tempo)
  for onset_vel, onset_time in zip(onset_velocities, onset_times):
    if onset_vel > velocity_threshold and onset_time >= start_time:  # filter quietest notes
      note_sequence.notes.add(
        instrument=9, pitch=42, is_drum=True,
        velocity=onset_vel,  # use fixed velocity here to avoid overfitting
        start_time=onset_time - start_time,
        end_time=onset_time - start_time)

  return note_sequence

# Allow encoding of a sequence that has no extracted examples
# by adding a quiet note after the desired length of time
def add_silent_note(note_sequence, num_bars):
  tempo = note_sequence.tempos[0].qpm
  length = 60/tempo * 4 * num_bars
  note_sequence.notes.add(
    instrument=9, pitch=42, velocity=0, start_time=length-0.02, 
    end_time=length-0.01, is_drum=True)
  
def get_bar_length(note_sequence):
  tempo = note_sequence.tempos[0].qpm
  return 60/tempo * 4

def sequence_is_shorter_than_full(note_sequence):
  return note_sequence.notes[-1].start_time < get_bar_length(note_sequence)

def get_rhythm_elements(y, sr):
  onset_env = librosa.onset.onset_strength(y, sr=sr)
  tempo = librosa.beat.tempo(onset_envelope=onset_env, max_tempo=180)[0]
  onset_times = librosa.onset.onset_detect(y, sr, units='time')
  onset_frames = librosa.onset.onset_detect(y, sr, units='frames')
  onset_strengths = librosa.onset.onset_strength(y, sr)[onset_frames]
  normalized_onset_strengths = onset_strengths / np.max(onset_strengths)
  onset_velocities = np.int32(normalized_onset_strengths * 127)

  return tempo, onset_times, onset_frames, onset_velocities

def make_tap_sequence(tempo, onset_times, onset_frames, onset_velocities,
                     velocity_threshold, start_time, end_time):
  note_sequence = music_pb2.NoteSequence()
  note_sequence.tempos.add(qpm=tempo)
  for onset_vel, onset_time in zip(onset_velocities, onset_times):
    if onset_vel > velocity_threshold and onset_time >= start_time and onset_time < end_time:  # filter quietest notes
      note_sequence.notes.add(
        instrument=9, pitch=42, is_drum=True,
        velocity=onset_vel,  # model will use fixed velocity here
        start_time=onset_time - start_time,
        end_time=onset_time -start_time + 0.01
      )
  return note_sequence

def audio_to_drum(f, velocity_threshold=30, temperature=1., force_sync=False, start_windows_on_downbeat=False):
  y, sr = librosa.load(f)
  # pad the beginning to avoid errors with onsets right at the start
  y = np.concatenate([np.zeros(1000),y])

  clip_length = float(len(y))/sr

  tap_sequences = []
  # Loop through the file, grabbing 2-bar sections at a time, estimating
  # tempos along the way to try to handle tempo variations

  tempo, onset_times, onset_frames, onset_velocities = get_rhythm_elements(y, sr)

  initial_start_time = onset_times[0]

  start_time = onset_times[0]
  beat_length = 60/tempo
  two_bar_length = beat_length * 8
  end_time = start_time + two_bar_length

  start_times = []
  lengths = []
  tempos = []

  start_times.append(start_time)
  lengths.append(end_time-start_time)
  tempos.append(tempo)

  tap_sequences.append(make_tap_sequence(tempo, onset_times, onset_frames, 
                       onset_velocities, velocity_threshold, start_time, end_time))

  start_time += two_bar_length; end_time += two_bar_length


  while start_time < clip_length:
    start_sample = int(librosa.core.time_to_samples(start_time, sr=sr))
    end_sample = int(librosa.core.time_to_samples(start_time + two_bar_length, sr=sr))
    current_section = y[start_sample:end_sample]
    tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(current_section, sr=sr), max_tempo=180)[0]

    beat_length = 60/tempo
    two_bar_length = beat_length * 8

    end_time = start_time + two_bar_length

    start_times.append(start_time)
    lengths.append(end_time-start_time)
    tempos.append(tempo)

    tap_sequences.append(make_tap_sequence(tempo, onset_times, onset_frames, 
                         onset_velocities, velocity_threshold, start_time, end_time))

    start_time += two_bar_length; end_time += two_bar_length
  
  # if there's a long gap before the first note, back it up close to 0
  def _shift_notes_to_beginning(s):
    start_time = s.notes[0].start_time
    if start_time > 0.1:
      for n in s.notes:
        n.start_time -= start_time
        n.end_time -=start_time
    return start_time
      
  def _shift_notes_later(s, start_time):
    for n in s.notes:
      n.start_time += start_time
      n.end_time +=start_time    
  
  def _sync_notes_with_onsets(s, onset_times):
    for n in s.notes:
      n_length = n.end_time - n.start_time
      closest_onset_index = np.argmin(np.abs(n.start_time - onset_times))
      n.start_time = onset_times[closest_onset_index]
      n.end_time = n.start_time + n_length
  
  drum_seqs = []
  for s in tap_sequences:
    try:
      if sequence_is_shorter_than_full(s):
        add_silent_note(s, 2)
        
      if start_windows_on_downbeat:
        note_start_time = _shift_notes_to_beginning(s)
      h = drumify(s, groovae_2bar_tap, temperature=temperature)
      h = change_tempo(h, s.tempos[0].qpm)
      
      if start_windows_on_downbeat and note_start_time > 0.1:
          _shift_notes_later(s, note_start_time)
        
      drum_seqs.append(h)
    except:
      continue  
      
  combined_tap_sequence = start_notes_at_0(combine_sequences_with_lengths(tap_sequences, lengths))
  combined_drum_sequence = start_notes_at_0(combine_sequences_with_lengths(drum_seqs, lengths))
  
  if force_sync:
    _sync_notes_with_onsets(combined_tap_sequence, onset_times)
    _sync_notes_with_onsets(combined_drum_sequence, onset_times)
  
  full_tap_audio = librosa.util.normalize(midi_synth.fluidsynth(combined_tap_sequence, sample_rate=sr))
  full_drum_audio = librosa.util.normalize(midi_synth.fluidsynth(combined_drum_sequence, sample_rate=sr))
  
  tap_and_onsets = mix_tracks(full_tap_audio, y[int(initial_start_time*sr):]/2, stereo=True)
  drums_and_original = mix_tracks(full_drum_audio, y[int(initial_start_time*sr):]/2, stereo=True)
  
  return full_drum_audio, full_tap_audio, tap_and_onsets, drums_and_original, combined_drum_sequence

In [ ]:
config_2_bar_humanize = configs.CONFIG_MAP['groovae_2bar_humanize']
groovae_2_bar_humanize = TrainedModel(config_2_bar_humanize, 1, checkpoint_dir_or_path=GROOVAE_2BAR_HUMANIZE)

def humanize(s, model, temperature=1.0):  
  encoding, mu, sigma = model.encode([s])
  decoded = model.decode(encoding, length=32, temperature=1.)[0]
  return change_tempo(decoded, s.tempos[0].qpm)

In [11]:
import random
tap_drums = [28, 31, 33, 37, 38, 40, 48, 50, 60]
bass_drums = [35, 36, 41, 43, 45,  47]
high_hats = [28, 31, 33, 39, 42, 44, 46, 49, 51, 52, 53, 57]
spcial_effects = [27, 29, 30, 32, 34, 54]

In [12]:
# Consts
peaks_times = [0.9287981859410431, 4.551111111111111, 8.17342403628118, 11.795736961451247, 15.418049886621315, 19.040362811791383, 22.66267573696145, 26.284988662131518, 29.88408163265306, 33.52961451247165]
inner_peaks_times = [1.3699773242630386, 1.8343764172335602, 2.7399546485260773, 3.2043537414965986, 3.645532879818594, 5.015510204081632, 6.362267573696145, 6.594467120181406, 6.826666666666667, 7.035646258503402, 7.267845804988662, 8.614603174603175, 9.079002267573696, 9.984580498866213, 10.448979591836734, 10.89015873015873, 12.236916099773243, 12.701315192743763, 13.606893424036281, 14.512471655328799, 15.882448979591837, 16.32362811791383, 17.252426303854875, 18.15800453514739, 20.851519274376418, 23.103854875283446, 23.54503401360544, 24.473832199546486, 24.91501133786848, 25.379410430839002, 25.820589569160997, 26.726167800453513, 27.190566893424037, 28.096145124716553, 28.53732426303855, 29.00172335600907, 29.442902494331065, 29.90730158730159, 30.34848072562358, 31.71845804988662, 32.182857142857145, 32.62403628117914, 33.99401360544218, 35.34077097505669, 35.57297052154195, 35.80517006802721, 36.24634920634921]
shuffled_peaks_times = [0.023219954648526078, 3.645532879818594, 7.267845804988662, 10.89015873015873, 14.512471655328799, 18.134784580498867, 21.757097505668934, 25.379410430839002, 29.00172335600907]
shuffled_inner_peaks_times = [0.46439909297052157, 0.905578231292517, 1.8343764172335602, 2.2755555555555556, 2.7399546485260773, 3.6687528344671203, 4.1099319727891155, 5.479909297052155, 5.688888888888889, 5.921088435374149, 6.15328798185941, 6.362267573696145, 7.7090249433106575, 8.17342403628118, 9.079002267573696, 9.543401360544218, 9.984580498866213, 11.354557823129252, 12.72453514739229, 13.630113378684808, 14.97687074829932, 15.418049886621315, 16.32362811791383, 17.22920634920635, 18.575963718820862, 19.040362811791383, 19.9459410430839, 20.410340136054423, 20.874739229024943, 21.78031746031746, 22.221496598639455, 23.56825396825397, 25.820589569160997, 26.261768707482993, 27.190566893424037, 27.631746031746033, 28.096145124716553, 29.466122448979593, 29.90730158730159, 30.812879818594105, 31.2540589569161, 31.71845804988662, 32.15963718820862, 32.60081632653061]

In [ ]:
import random
final_drums = music_pb2.NoteSequence()

bass_pitch = bass_drums[0]
bass_pitch_1 = bass_drums[1]
hh_pitch_1 = high_hats[3]
hh_pitch = high_hats[5]
tap_pitch = tap_drums[7]

for i, p in enumerate(peaks_times):  
  final_drums.notes.add(pitch=bass_pitch, start_time=p-0.05, end_time=p+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=bass_pitch, start_time=p-0.048, end_time=p+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=bass_pitch, start_time=p-0.05, end_time=p+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=hh_pitch_1, start_time=p-0.048, end_time=p+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  if i % 5:
    se = random.choice(spcial_effects)
    final_drums.notes.add(pitch=se, start_time=p-0.05, end_time=p+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  i += 1


for i, ip in enumerate(inner_peaks_times):
  final_drums.notes.add(pitch=tap_pitch, start_time=ip-0.05, end_time=ip+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=tap_pitch, start_time=ip-0.048, end_time=ip+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=hh_pitch, start_time=ip-0.05, end_time=ip+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=hh_pitch, start_time=ip-0.048, end_time=ip+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=hh_pitch, start_time=ip-0.05, end_time=ip+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  if i % 4 == 0:
    final_drums.notes.add(pitch=bass_pitch, start_time=ip-0.048, end_time=ip+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
    final_drums.notes.add(pitch=bass_pitch, start_time=ip-0.05, end_time=ip+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  i += 1

final_drums.total_time = max(peaks_times[-1], inner_peaks_times[-1])+0.1


mean = np.mean([peaks_times[i] - peaks_times[i-1] for i in range(1, len(peaks_times))])
constant_p = [peaks_times[0] + (i * mean)/4 for i in range(len(peaks_times) * 4)]
for m in constant_p: 
  final_drums.notes.add(pitch=bass_pitch_1, start_time=m-0.05, end_time=m+1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))
  final_drums.notes.add(pitch=bass_pitch_1, start_time=m-0.05, end_time=m+0.1, is_drum=True, instrument=np.random.randint(8, 12), velocity=np.random.randint(50, 110))


final_drums.tempos.add(qpm=60)

# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(final_drums)

# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(final_drums,synth=note_seq.fluidsynth)
# note_seq.sequence_proto_to_midi_file(drums, "midi_final.mid")
# print(final_drums)

In [13]:
temperature = 1.32
velocity_threshold = 0.05
stereo = True

f = '/content/samp_2.wav'
print("\n\n\nPlaying %s: " %(f))
y,sr = librosa.load(f)
IPython.display.display(IPython.display.Audio(y, rate=sr))
full_drum_audio, full_tap_audio, tap_and_onsets, drums_and_original, combined_drum_sequence = audio_to_drum(f, velocity_threshold=velocity_threshold, temperature=temperature)

# new_beats.append(combined_drum_sequence)
# new_drum_audios.append(full_drum_audio)
# combined_audios.append(drums_and_original)
print("Playing the rhythm detected in %s: " %(f))
IPython.display.display(IPython.display.Audio(full_tap_audio, rate=sr))
print("Playing drums generated from %s: " %(f))
IPython.display.display(IPython.display.Audio(full_drum_audio, rate=sr))
print("Playing %s together with drums" %(f))
IPython.display.display(IPython.display.Audio(drums_and_original, rate=sr))




Playing /content/samp_2.wav: 


AttributeError: ignored